In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------

update 2020/02/10 update output 11-12 input 8-9

Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 12

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v09"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v09 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2020M02D10 UTC 16:30


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111011_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//145102_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//131902_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142625_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155562_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132895_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153021_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115801_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127443_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122784_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//172497_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//181976_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232144_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//223940_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127450_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//145109_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122810_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111019_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232160_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153028_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//224002_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132930_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//158116_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//181993_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//131909_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//172524_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//115809_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142634_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155579_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153035_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//224009_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132947_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111045_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//172541_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142642_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127468_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//158143_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122846_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//116006_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232250_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132170_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//184000_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//127485_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//225117_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132955_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//172548_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155617_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153043_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111070_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//158211_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122854_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//117102_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142651_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232303_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132224_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//162461_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//142658_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//111087_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//155635_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//211033_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//153050_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//122870_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//158219_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//232401_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132241_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//145602_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//117109_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//225170_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//132973_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//162468_V12.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//462169_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//562946_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//616805_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//633096_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//491209_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//564907_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//567533_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//445814_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//456203_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//622652_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//453188_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//474011_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//612042_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//624452_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//453250_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//474042_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//612049_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//622659_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//456206_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//562982_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//491403_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//561490_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//521806_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//621002_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//624459_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//565017_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//462179_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//567560_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//612093_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//567604_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//561506_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//491503_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//453295_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//624467_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//475101_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//565025_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//634140_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//456305_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//562988_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//445830_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//622212_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//521904_V12.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//613011_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//453304_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//523000_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//634193_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//456420_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//562996_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//567706_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//445847_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//622219_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//565036_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//624475_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//475109_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//622676_V12.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V12//491605_V12.csv
/volumes/data/Y2018M

OSError: [Errno 28] No space left on device

In [ ]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [ ]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932  
0:23:35.132388

